In [3]:
import os
import json
import re
import pickle
from dotenv import load_dotenv
from collections import defaultdict

from get_data_from_or import get_client
import openreview
from datetime import datetime


In [2]:
load_dotenv("credentials.env")
client = get_client(os.getenv("OR_API_VERSION"))

1


In [3]:
notes = client.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission', details="directReplies")

In [4]:
for note in notes:
    for reply in note.details["directReplies"]:
        if reply["invitation"].endswith("Official_Review"):
            for signature in reply["signatures"]:
                if signature.startswith("~"):
                    print(json.dumps(reply, indent=3))
                    break

1000


In [5]:
venues = client.get_group(id='venues').members

In [6]:
print(len(venues))

1150


Single Blind submission which have peer reviews

In [7]:
data = {}
for venue in venues:
    data[venue] = client.get_all_notes(invitation=f"{venue}/-/Submission", details="directReplies")

In [8]:
peer_reviews = defaultdict(dict)
for venue in data:
    for paper in data[venue]:
        peer_reviews[venue][paper.id] = []
        for reply in paper.details["directReplies"]:
            if reply["invitation"].endswith("Official_Review"):
                peer_reviews[venue][paper.id].append(reply)


In [9]:
with open("open_review_reviews_20_12_2023.json", "w") as outfile:
    json.dump(peer_reviews, outfile, indent=2)

In [13]:
papers = defaultdict(dict)
json_data = defaultdict(dict)
for venue in data:
    if len(data[venue])==0:
        continue
    else:
        for paper in data[venue]:
            papers[venue][paper.id]=paper
            json_data[venue][paper.id]=paper.to_json()

In [14]:
with open("open_review_papers_plus_reviews_20_12_2023.json", "w") as outfile:
    json.dump(json_data, outfile, indent=2)

Double Blind submission which have peer reviews

In [15]:
data2 = {}
for venue in venues:
    data2[venue] = client.get_all_notes(invitation=f"{venue}/-/Blind_Submission", details="directReplies")

Getting V1 Notes: 100%|█████████▉| 1044/1046 [00:00<00:00, 1337.93it/s]


In [16]:
peer_reviews2 = defaultdict(dict)
for venue in data2:
    for paper in data2[venue]:
        peer_reviews2[venue][paper.id] = []
        for reply in paper.details["directReplies"]:
            if reply["invitation"].endswith("Official_Review"):
                peer_reviews2[venue][paper.id].append(reply)


In [17]:
with open("open_review_reviews_20_12_2023_double_blind.json", "w") as outfile:
    json.dump(peer_reviews2, outfile, indent=2)

In [18]:
papers2 = defaultdict(dict)
json_data2 = defaultdict(dict)
for venue in data2:
    if len(data2[venue])==0:
        continue
    else:
        for paper in data2[venue]:
            papers2[venue][paper.id]=paper
            json_data2[venue][paper.id]=paper.to_json()

In [19]:
with open("open_review_papers_plus_reviews_20_12_2023_double_blind.json", "w") as outfile:
    json.dump(json_data2, outfile, indent=2)

In [20]:
n_venues_sb=len([j for j in json_data if len(json_data[j])>0])
n_venues_db=len([j for j in json_data2 if len(json_data2[j])>0])
n_reviews_sb=0
n_papers_sb=0
n_reviews_db=0
n_papers_db=0
n_reviews_open_pa_sb=0
n_reviews_open_pa_db=0
non_anonymous_reviewers=defaultdict(int)
for v in peer_reviews.values():
    for p in v.values():
        if len(p)>0:
            n_papers_sb+=1
        for r in p:
            n_reviews_sb+=1
            identity=None
            for signature in r["signatures"]:
                if signature.startswith("~") or "@" in signature:
                    identity=signature
            if identity is not None:
                non_anonymous_reviewers[identity]+=1
                n_reviews_open_pa_sb+=1
                
for v in peer_reviews2.values():
    for p in v.values():
        if len(p)>0:
            n_papers_db+=1
        for r in p:
            n_reviews_db+=1
            identity=None
            for signature in r["signatures"]:
                if signature.startswith("~") or "@" in signature:
                    identity=signature
            if identity is not None:
                non_anonymous_reviewers[identity]+=1
                n_reviews_open_pa_db+=1


In [21]:

print("Some stats:")
print(f"venues with single blind reviews: {n_venues_sb}")
print(f"venues with double blind reviews: {n_venues_db}")
print(f"total venues: {n_venues_db+n_venues_sb}")
print()
print(f"submissions single blind: {n_papers_sb}")
print(f"submissions double blind: {n_papers_db}")
print(f"total submissions: {n_papers_db+n_papers_sb}")
print()
print(f"reviews single blind: {n_reviews_sb}")
print(f"reviews double blind: {n_reviews_db}")
print(f"total reviews: {n_reviews_db+n_reviews_sb}")
print()
print(f"avg. reviews/submission single blind: {n_reviews_sb/n_papers_sb:.2f}")
print(f"avg. reviews/submission double blind: {n_reviews_db/n_papers_db:.2f}")
print(f"total avg. reviews/submission: {(n_reviews_db+n_reviews_sb)/(n_papers_db+n_papers_sb):.2f}")
print()
print(f"post acceptance deanonymized reviews single blind: {n_reviews_open_pa_sb}")
print(f"post acceptance deanonymized reviews double blind: {n_reviews_open_pa_db}")
print(f"total post acceptance deanonymized reviews: {n_reviews_open_pa_db+n_reviews_open_pa_sb}")


Some stats:
venues with single blind reviews: 95
venues with double blind reviews: 277
total venues: 372

submissions single blind: 1809
submissions double blind: 22902
total submissions: 24711

reviews single blind: 5265
reviews double blind: 80211
total reviews: 85476

avg. reviews/submission single blind: 2.91
avg. reviews/submission double blind: 3.50
total avg. reviews/submission: 3.46

post acceptance deanonymized reviews single blind: 218
post acceptance deanonymized reviews double blind: 0
total post acceptance deanonymized reviews: 218


In [22]:
non_anonymous_reviewer_profiles = openreview.tools.get_profiles(
    client,
    ids_or_emails=list(non_anonymous_reviewers.keys())
)


In [23]:
for conf in json_data2:
    print(conf)

ICLR.cc/2018/Conference
ICLR.cc/2019/Conference
ICLR.cc/2019/Workshop/DeepGenStruct
ICLR.cc/2019/Workshop/LLD
ICLR.cc/2019/Workshop/RML
ICLR.cc/2019/Workshop/drlStructPred
ICLR.cc/2020/Conference
ICLR.cc/2020/Workshop/DeepDiffEq
ICLR.cc/2021/Conference
ICLR.cc/2021/Workshop/EBM
ICLR.cc/2021/Workshop/GTRL
ICLR.cc/2021/Workshop/Learning_to_Learn
ICLR.cc/2021/Workshop/Neural_Compression
ICLR.cc/2021/Workshop/SSL-RL
ICLR.cc/2022/Conference
ICLR.cc/2022/Track/Blog
ICLR.cc/2022/Workshop/ALOE
ICLR.cc/2022/Workshop/AfricaNLP
ICLR.cc/2022/Workshop/Cells2Societies
ICLR.cc/2022/Workshop/DGM4HSD
ICLR.cc/2022/Workshop/DL4C
ICLR.cc/2022/Workshop/DLG4NLP
ICLR.cc/2022/Workshop/EmeCom
ICLR.cc/2022/Workshop/GMS
ICLR.cc/2022/Workshop/GPL
ICLR.cc/2022/Workshop/GTRL
ICLR.cc/2022/Workshop/MLDD
ICLR.cc/2022/Workshop/OSC
auai.org/UAI/2021/Workshop/TPM
auai.org/UAI/2022/Conference
auai.org/UAI/2022/Workshop/CRL
ICML.cc/2018/Workshop/NAMPI
ICML.cc/2019/Workshop/AMTL
ICML.cc/2019/Workshop/Deep_Phenomena
ICML.cc/

In [24]:
for conf in json_data2:
    for paper in json_data2[conf]:
        dt_object = datetime.fromtimestamp(json_data2[conf][paper]["cdate"]/1000.0)
        print(dt_object.strftime("%Y-%m-%d %H:%M:%S"))
        print(json.dumps(json_data2[conf][paper], indent=3))
        break
    break

2018-02-15 22:29:49
{
   "id": "ryzm6BATZ",
   "original": "B1zmTBAT-",
   "cdate": 1518730189355,
   "pdate": null,
   "odate": null,
   "mdate": null,
   "tcdate": 1508953370259,
   "tmdate": 1644493022819,
   "ddate": null,
   "number": 93,
   "content": {
      "title": "Image Quality Assessment Techniques Improve Training and Evaluation of Energy-Based Generative Adversarial Networks",
      "abstract": "We propose a new, multi-component energy function for energy-based Generative Adversarial Networks (GANs) based on methods from the image quality assessment literature. Our approach expands on the Boundary Equilibrium Generative Adversarial Network (BEGAN) by outlining some of the short-comings of the original energy and loss functions. We address these short-comings by incorporating an l1 score, the Gradient Magnitude Similarity score, and a chrominance score into the new energy function. We then provide a set of systematic experiments that explore its hyper-parameters. We show t

In [25]:
peer_reviews.update(peer_reviews2)
papers.update(papers2)

In [27]:
with open('papers.pickle', 'wb') as f:
    pickle.dump(papers, f, pickle.HIGHEST_PROTOCOL)
with open('peer_reviews.pickle', 'wb') as f:
    pickle.dump(peer_reviews, f, pickle.HIGHEST_PROTOCOL)

In [28]:
def get_year(conf_name):
    sub_names = conf_name.split("/")
    for name in sub_names:
        if len(name)==4 and name[:2] in ["19","20"] and name.isnumeric():
            return name
        
def get_conference(conf_name):
    conf_name = conf_name.split("/")[0]    
    return conf_name
    
def get_decision(paper):
    for reply in paper.details["directReplies"]:
        if reply["invitation"].endswith("Decision"):
            return reply["content"]["decision"]

def get_meta_review(paper):
    for reply in paper.details["directReplies"]:
        if reply["invitation"].endswith("Meta_Review"):
            return reply

def get_avg_score(reviews):
    return sum([r["score"] for r in reviews if r["score"] is not None])

def get_meta_score(reviews):
    return

def get_score(review, keys):
    if review is not None:
        score = None
        for kw in [
            "rating", 
            "final_rating", 
            "final_rating_after_the_rebuttal", 
            "overall_assessment",
            "overall_rating", 
            "review_rating", 
            "score",
            "Overall Score",
            "Overall score",
            "Q6 Overall score",
            "recommendation"]:
            if kw in review["content"]:
                # if kw=="Q6 Overall score":
                    # print(re.match("-?\d+",review["content"][kw]).group(0))
                try:
                    score = re.match("-?\d+",review["content"][kw])
                    if score is not None:
                        break
                except TypeError:
                    continue
        if score is not None:
            return int(score.group(0))
        else:
            keys|=keys|review["content"].keys()
review_keys=set()
confs_by_year = defaultdict(lambda: defaultdict(list))
for conf in peer_reviews:
    for paper in peer_reviews[conf]:
        decision = get_decision(papers[conf][paper])
        for pr in peer_reviews[conf][paper]:
            score = get_score(pr, review_keys)
            pr["score"] = score
        if decision is not None: 
            if len(peer_reviews[conf][paper])>0:
                confs_by_year[get_conference(conf)][get_year(conf)].append(
                    {
                        "decision": decision,
                        "avg_score": get_avg_score(peer_reviews[conf][paper]),
                        "reviews": peer_reviews[conf][paper]
                    }
                )
    

In [31]:
review_keys

{'Brief summary',
 'Conflicts',
 'Detailed comments',
 'Justification for rating',
 'Overall Merit Score',
 'Overall Score',
 'Paper strengths',
 'Paper weaknesses',
 'Poster (if paper is rejected)',
 'Review Inclusion',
 'Reviewer Confidence',
 'Sufficiently Alt',
 '[Optional] Respond to feedback request by the authors',
 'accessibility',
 'anonymity_and_confidentiality',
 'awarded badges',
 'best_paper_award',
 'category',
 'clarity',
 'clarity_comment',
 'clarity_of_presentation',
 'comment',
 'confidence',
 'deanonymize_review',
 'decision',
 'detailed comments',
 'detailed_comments',
 'evaluation',
 'expertise',
 'grade',
 'groundsforrejection',
 'impact',
 'importance',
 'importance_comment',
 'intersection',
 'intersection_comment',
 'issues',
 'justification_of_the_preliminary_rating',
 'litreview',
 'novelty',
 'originality',
 'paper_type',
 'preliminary_rating',
 'problemstatement',
 'quality_of_the_limitations_section',
 'questions_for_rebuttal',
 'questions_to_address_in_th

In [32]:
with open('confs_by_year.pickle', 'wb') as f:
    pickle.dump(dict(confs_by_year), f, pickle.HIGHEST_PROTOCOL)

In [30]:
print(json.dumps(non_anonymous_reviewer_profiles[0].to_json(), indent=3))

{
   "id": "~Abraham_Nunes1",
   "tcdate": 1597167128053,
   "tmdate": 1626094184532,
   "referent": null,
   "packaging": null,
   "invitation": "~/-/profiles",
   "readers": [
      "everyone"
   ],
   "nonreaders": null,
   "signatures": [
      "~Abraham_Nunes1"
   ],
   "writers": [
      "OpenReview.net"
   ],
   "content": {
      "preferredEmail": "****@dal.ca",
      "homepage": "http://www.abrahamnunes.com",
      "emails": [
         "****@dal.ca"
      ],
      "names": [
         {
            "first": "Abraham",
            "middle": "",
            "last": "Nunes",
            "preferred": true,
            "username": "~Abraham_Nunes1",
            "fullname": "Abraham Nunes"
         }
      ],
      "history": [
         {
            "position": "Psychiatry resident",
            "start": 2014,
            "end": 2021,
            "institution": {
               "domain": "dal.ca",
               "name": "Dalhousie University"
            }
         },
         {
   

# API V2

In [1]:
client2 = get_client("2")
venue_group = client2.get_group("ICLR.cc/2022/Conference")
print(venue_group.to_json().keys())

NameError: name 'get_client' is not defined

In [9]:
print(venue_group.to_json()["readers"])

['everyone']


In [10]:
json_data3 = {}
for venue_id in venues:
    try:
        venue_group = client.get_group(venue_id)
        submission_name = venue_group.content['submission_name']['value']
        submissions = client.get_all_notes(invitation=f'{venue_id}/-/{submission_name}', details='replies')
        for paper in submissions:
            json_data3[venue_id][paper.id]=paper.to_json()
        print(f"{venue_id} is API V2 venue.")
    except AttributeError:
        pass
    except openreview.OpenReviewException as e:
        print(e)

NameError: name 'venues' is not defined

In [ ]:
json_data3

In [ ]:
venues_raw = client.get_group(id='venues').to_json()

In [ ]:
venues_raw.keys()